# Motion Compensated Discrete Wavelet Transform (MCDWT)

MCDWT is a multiresolution [video] transform. The input sequence of [pixels] are [decorrelated] in the [time] and in the [spatial] domains. The output sequence of [coefficients] have a smaller [entropy] than the original pixels, and the information is represented by temporal and spatial [resolution levels].

* In MCDWT (and in general, in video coding) temporal decorrelation is provided by [MC (Motion Compensation)], where the [prediction images] are generated with an algorithm that uses only the information available at the [decoder]. 

* Spatial decorrelation is performed by the [analysis filtering] used in the [2D-DWT].

[video]: https://en.wikipedia.org/wiki/Video
[decorrelator]: https://en.wikipedia.org/wiki/Decorrelation
[visual]: https://en.wikipedia.org/wiki/Visual_perception
[pixels]: https://en.wikipedia.org/wiki/Pixel
[decorrelated]: https://en.wikipedia.org/wiki/Decorrelation
[time]: https://en.wikipedia.org/wiki/Time_domain
[spatial]: https://www.quora.com/What-is-spatial-domain-in-image-processing
[coefficients]: https://www.quora.com/What-is-spatial-domain-in-image-processing
[entropy]: https://en.wikipedia.org/wiki/Entropy
[resolution levels]: https://en.wikipedia.org/wiki/Image_resolution
[MC (Motion Compensation)]: https://en.wikipedia.org/wiki/Motion_compensation
[decoder]: https://en.wikipedia.org/wiki/Decoder
[prediction images]: https://en.wikipedia.org/wiki/Decoder
[analysis filtering]: https://en.wikipedia.org/wiki/Digital_filter#Analysis_techniques
[2D-DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform
[EBCOT]: http://nptel.ac.in/courses/117105083/pdf/ssg_m5l15.pdf

## 1. Video [scalability]

MCDWT inputs a [video] and outputs a video, in a way that when only a portion of the data of the transformed video is used, a video with a lower [temporal resolution], lower [spatial resolution] or/and lower
[quality] can be generated.

If all the transformed data is used, then the original video is restored (MCDWT is a lossless transform). The forward transform's output has exactly the same number of elements than the input video (for example, no extra motion fields are produced). At this time, we will focuse only on spatial [scalability]. Other types of scabilities will be addressed later.

[temporal resolution]: https://en.wikipedia.org/wiki/Temporal_resolution
[spatial resolution]: https://en.wikipedia.org/wiki/Image_resolution#Spatial_resolution
[quality]: https://en.wikipedia.org/wiki/Compression_artifact
[scalability]: http://inst.eecs.berkeley.edu/~ee290t/sp04/lectures/videowavelet_UCB1-3.pdf
[video]: https://en.wikipedia.org/wiki/Video

## 2. Video transformation alternatives

To obtain a multiresolution version or a video, the [DWT] (Discrete
Wavelet Transform) can be applied along temporal (t) and
spatial domains (2D). At this point, two different alternatives
are available: (1) a t+2D transform or (2) a 2D+t
transform. In a t+2D transform, the video is first analyzed
over the time domain and next, over the space domain. A 2D+t
transform does just the opposite.

[DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform

Each choice has a number of *pros* and *cons*. For example, in a
t+2D transform we can apply directly any image predictor based
on motion estimation because the input is a normal video. However, if
we implement a 2D+t transform, the input to the motion estimator is a sequence of images in the DWT domain. [The overwhelming majority of DWT's][Friendly Guide] are not [shift invariant], which basically means $\text{DWT}(s[t]) \neq \text{DWT}(s[t+x])$, where $x$ is a
displacement of $s[t]$ along the time domain. Therefore, motion estimators which compare pixel values will not work on the DWT domain. On the other hand, if we want to provide true spatial scalability (processing only those spatial resolutions (scales) necessary to get a spatially scaled of our video), a t+2D transformed video is not suitable because the first step of the forward transform (t) should be reversed at full resolution in the backward transform (as the forward transform did).

[shift invariant]: http://www.polyvalens.com/blog/wavelets/theory
[Friendly Guide]: http://www.polyvalens.com/blog/wavelets/theory

## 3. Wavelet and pyramid domains

Indeed, the DWT allows to get a scalable representation of a image, and by extension of a video, when we apply the DWT on all the images of the video. However, this can be also done with [Gaussian and Laplacian pyramids][Laplacian Pyramids]. Image pyramids are interesting because they are shift invariant and therefore, one can operate within the scales as they are *normal* images. Unfortunately, as a consecuence of pyramids representations are not critically sampled, they need more picture elements than in [wavelet pyramids][Wavelet Pyramids], and this is a serious drawback when compressing. Luckily, it is very fast to convert a Laplacian pyramid representation into it DWT equivalent representation, and viceversa. For this reason, even if we use wavelet pyramids in our images, we can suppose at any moment that we are working with the Laplacian pyramid of those images.

[Laplacian Pyramids]: https://en.wikipedia.org/wiki/Pyramid_(image_processing)
[Wavelet Pyramids]: http://www.vtvt.ece.vt.edu/research/references/video/DCT_Video_Compression/Zhang92a.pdf

## 4. The \texttt{K}-levels 2D Discrete Wavelet Transform (2D-DWT) of a video

As said before, a [2D-DWT][2D-DWT] (2 Dimensions - Discrete Wavelet Transform) generates a scalable representation of an image and by extension, of a video if we apply the DWT to all the images of the video. This is done, for example, in the [motion JPEG2000 video compression standard][J2K]. Notice that only the spatial redundancy is exploited. All the temporal redundancy is still in the video.

[J2K]: https://en.wikipedia.org/wiki/JPEG_2000
[2D-DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform

### Input

A sequence $s$ of $N$ images:

```
                                                      x 
+---------------+  +---------------+     +---------------+
|               |  |               |     |            |  |
|               |  |               |   y |----------- o <---- s[N-1][y][x]
|               |  |               | ... |               |
|               |  |               |     |               |
|               |  |               |     |               |
|               |  |               |     |               |
+---------------+  +---------------+     +---------------+
      s[0]               s[1]                 s[N-1]
```

### Output

A sequence $S$ of $N$ pyramids. For example, a ($K=2$)-levels, the 2D-DWT looks like:

```
+---+---+-------+  +---+---+-------+     +---+---+-------+
|LL2|HL2|       |  |   |   |       |     |   |   |       |
+---+---+  HL1  |  +---+---+       |     +---+---+       |
|LH2|HH2|       |  |   |   |       |     |   |   |       |
+---+---+-------+  +---+---+-------+ ... +---+---+-------+
|       |       |  |       |       |     |       |       |
|  LH1  |  HH1  |  |       |       |     |       |       |
|       |       |  |       |       |     |       |       |        
+-------+-------+  +-------+-------+     +-------+-------+
      S[0]               S[1]                  S[N-1]
```
where $L$ and $H$ stands for *low-pass filtered* and *high-pass filtered*, respectively. The integer > 1 that follows these letters represents the subband level. For the sake of simplicity, we will denote the subbands $\{LH^k, HL^k, HH^k\}$ as only $H^k$, and $LL^k$ as only $L^k$. 

### Algorithm
\lstinputlisting[language=python]{src/sequence/2D_DWT.py}
```python
def sequence_2D_DWT(s, K):
    for image in s:
        _2D_DWT(image, K) # In-place computation
    return s
    
S = sequence_2D_DWT(s, K)
```

### Inverse algorithm
\lstinputlisting[language=python]{src/sequence/2D_iDWT.py}

```python
def sequence_2D_iDWT(S, K):
    for pyramid in S:
        _2D_iDWT(pyramid, K) # In-place computation
    return S
    
s = sequence_2D_iDWT(S, K)
```

### Scalability

The 2D-DWT applied to a video produces a scalable representation in the space (we can extract different videos with different spatial scales or resolutions), in the time (we can extract diferent videos with different number of frames), and in quality (we can get the DWT
coefficients with different quantization steps to reconstruct videos of different quality). In the last example, subbands $\{S_0.LL^2, S_1.LL^2, ..., S_{N-1}.LL^2\}$ represent the scale (number) 2 of the original video (the spatial resolution of this is the resolution of $s$ divided by 4 in each spatial dimension).

For example, to reconstruct the scale 1 (which is provided by the subbands $\{LL^2, LH^2, HL^2, HH^2\}$), we must apply the 2D-iDWT (1-levels 2D inverse DWT).

### Implementation of 2D-DWT and 2D-iDWT

See for example, [pywt.wavedec2()](https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2) at [PyWavelets](https://pywavelets.readthedocs.io/en/latest/index.html).

## 5. Redundancy and compression

The 2D-DWT incorporates to \texttt{S} an interesting feature: usually, \texttt{H} subbands has a lower entropy than \texttt{s}. This means that if we apply to \texttt{S} an entropy encoder, we can get a shorter representation of the video than if we encode \texttt{s} directly. This is a consequence of that 2D-DWT removes (most of) the spatial redudancy of the images of the video (neighboring pixels tend to have similar values and their substraction tends to produce zeros).

## 6. Why Motion Compensated DWT (MCDWT)?

As we have said, the 2D-DWT does not exploit the temporal redundancy of a video. Consequently, we can achieve higher compression ratios (in addition to the 2D-DWT) if we apply a 1D-DWT along the temporal domain. This is exactly what MCDWT does. However, due to the temporal redundancy is generated mainly by the presence of objects in the scene of the video which are moving with respect to the camera, some sort of motion compensation should be used before substacting the prediction images to the original ones.


## 7. Forward (direct) MCDWT (1-levels) butterfly

MCDWT is based on the idea that, to provide MC without transmitting the motio information from the encoder to the decoder, the motion information is computed at both ends, using the information that is available at the decoder.

### Input

Three frames of $s$: $a$, $b$ and $c$.

### Output

Three frames $A=\{A.L, A.H\}$ (intra-coded), $B=\{B.L, \tilde{B}.H\}$ (bidirectionally predicted) and $C=\{C.L, C.H\}$ (intra-coded).

### Algorithm

![MCDWT](forward.png)

```python
A.L, A.H = 2D_DWT(a)
B.L, B.H = 2D_DWT(b)
C.L, C.H = 2D_DWT(c)
[A.L] = 2D_iDWT(A.L,0) # Interpolate low-freqs
[A.H] = 2D_iDWT(0,A.H) # Interpolate high-freqs
[B.L] = 2D_iDWT(B.L,0)
[B.H] = 2D_iDWT(0,B.H)
[C.L] = 2D_iDWT(C.L,0)
[C.H] = 2D_iDWT(0,C.H)
[B.L]->[A.L] = ME([B.L], [A.L]) # Motion estimation
[B_A.H] = MC([A.H], [B.L]->[A.L]) # Project [A.H] using [B.L]->[A.L]
[B.L]->[C.L] = ME([B.L], [C.L])
[B_C.H] = MC([C.H], [B.L]->[C.L])
[~B.H] = [B.H] - [B_A.H]/2 + [B_C.H]/2
~B.H = 2D_DWT([~B.H])
A = A.L, A.H
B = B.L, ~B.H
C = C.L, C.H
```

Notice that, even if $a$ and $c$ (at full resolution) are available at the decoder, only $B.L$ is. So, does not make sense to search the low resolution structures of $B.L$ in the high resolution images $a$ and $c$, because even if a perfect match is achieved, the prediction error would be different than $0$ as a consequence of the high resolution information is missing in $B.L$.

<a id='Another_example'></a>
## Another example (only butterfly)

Lets suppose that we have procesed the sequence of images one step and a second one is performed on the output of this first step. At this moment, the structure of the images would be:

<img src="graphics/after_inverse_step.svg">

### Analyze the input images

\begin{equation}
  \{A.L^2, A.H^2\} = \text{DWT}(A.L)\\
  \{B.L^2, B.H^2\} = \text{DWT}(B.L)\\
  \{C.L^2, C.H^2\} = \text{DWT}(C.L)
\end{equation}

### Zoom-in $L$ subbands

\begin{equation}
  [A.L^2] = \text{iDWT}(A.L^2, 0)\\
  [B.L^2] = \text{iDWT}(B.L^2, 0)\\
  [C.L^2] = \text{iDWT}(C.L^2, 0)
\end{equation}

### Merge and zoom-in $H$ subbands

\begin{equation}
  [A.H^2] = \text{iDWT}(0, A.H^2)\\
  [B.H^2] = \text{iDWT}(0, B.H^2)\\
  [C.H^2] = \text{iDWT}(0, C.H^2)
\end{equation}

### Generate a prediction $[\hat{B}.H^2]$¡

\begin{equation}
  [B_A.H^2] = [A.H^2]\overset{[A.L^2]\rightarrow[B.L^2]}{\longrightarrow}[B.H^2]\\
  [B_C.H^2] = [C.H^2]\overset{[C.L^2]\rightarrow[B.L^2]}{\longrightarrow}[B.H^2]\\  
\end{equation}

Note: predictions do not need to be exhaustive. Depending on the quality of the predictions for $[B.L^2]$:

\begin{equation}
  [B_A.L^2] = [A.L^2]\overset{[A.L^2]\rightarrow[B.L^2]}{\longrightarrow}[B.L^2]\\
  [B_C.L^2] = [C.L^2]\overset{[C.L^2]\rightarrow[B.L^2]}{\longrightarrow}[B.L^2],\\  
\end{equation}

comparing $[B_A.L^2]$ and $[B_C.L^2]$ with $[B.L^2]$. In fact, $[B_A.H^2]$ and $[B_C.H^2]$ could have "holes" (regions to 0, supposing that the pixels are positive), producing a parcial prediction $[\hat{B}.H^2]$ for $[B.H^2]$. In the simplest case

\begin{equation}
  [\hat{B}.H^2] = \frac{[B_A.H^2] + [B_C.H^2]}{2}.
\end{equation}

Finally, (as in the forward transform) the prediction should take into cosideration how to minimize the energy of the residue

\begin{equation}
  [\tilde{B}.L^2] = [B.L^2] - [\hat{B}.L^2].
\end{equation}

### Compute the residue $[\tilde{B}.H^2]$

\begin{equation}
  [\tilde{B}.H^2] = [B.H^2] - [\hat{B}.H^2]
\end{equation}

### Unmerge $\tilde{B}.H^2$ subbands

\begin{equation}
  \tilde{B}.H^2 = \text{DWT}(0, [\tilde{B}.H^2])
\end{equation}

### Compose the residue in the wavelet domain

\begin{equation}
   \{B.L^2, \tilde{B}.H^2\}
\end{equation}

<img src="graphics/before_inverse_step.svg">

## 8. Backward (inverse) MCDWT butterfly

#### Input

* Three frames $A=\{A.L, A.H\}$, $B=\{B.L, \tilde{B}.H\}$, and $C=\{C.L, C.H\}$. 

#### Output

* Three frames $a$, $b$ and $c$.

#### Algorithm

![MCDWT](backward.png)


## Example continuation (only one butterfly)

Lets restore the structure for the three transformed images of [the previous example](#Another_example).

<img src="graphics/before_inverse_step.svg">

### Zoom-in $L$ subbands

\begin{equation}
  [A.L^2] = \text{iDWT}(A.L^2, 0)\\
  [B.L^2] = \text{iDWT}(B.L^2, 0)\\
  [C.L^2] = \text{iDWT}(C.L^2, 0)
\end{equation}

### Merge (and zoom-in) $H$ subbands

\begin{equation}
  [A.H^2] = \text{iDWT}(0, A.H^2)\\
  [\tilde{B}.H^2] = \text{iDWT}(0, \tilde{B}.H^2)\\
  [C.H^2] = \text{iDWT}(0, C.H^2)
\end{equation}

### Generate a prediction $[\hat{B}.H^2]$

Identical to the prediction step performed at the forward transform.

### Restore the original subband $[B.H^2]$

\begin{equation}
  [B.H^2] = [\hat{B}.H^2] + [\tilde{B}.H^2]
\end{equation}

### Compute $L$ subbands

\begin{equation}
  A.L = [A.L^2] + [A.H^2]\\
  B.L = [B.L^2] + [B.H^2]\\
  C.L = [C.L^2] + [C.H^2]
\end{equation}

<img src="graphics/after_inverse_step.svg">

## $K$-levels (forward) MCDWT

#### MCDWT input

* A sequence $s$ of $N$ images.

#### MCDWT output

* A sequence of $K+1$ temporal subbands, where each subband is a sequence of $N/2^K$ (wavelet) pyramids.

#### Algorithm

<img src="mcdwt_representation.svg">

### Data extraction examples

#### Temporal scalability

* Scale 2:

  $
    s_0.L^0 = \text{DWT}^{-2}(S_0) = \text{DWT}^{-1}(\text{DWT}^{-1}(s_0.L^2, s_0.H^2), s_0.H^1),\\
    s_4.L^0 = \text{DWT}^{-2}(S_4),\\
    s_2.L^0 = \text{MCDWT}^{-2}(S_0, S_2, S_4) = (s_2.L^1 = \text{MCDWT}^{-1}(\{s_0.L^2,s_0.H^2\}, \{s_4.L^2, s_4.H^2\}, \{s_2.L^2, \tilde{s}_2.H^2\})) 
  $

#### Spatial scalability

* Scale 2:

  $
    s_0.L^2, s_1.L^2, s_2.L^2, s_3.L^2, s_3.L^2.
  $
 
* Scale 1:

  $
    s_0.L^1 = DWT^{-1}(s_0.L^2, s_0,H^2), \\
    s_4.L^1 = DWT^{-1}(s_4.L^2, s_4.H^2), \\
    s_2.L^1 = MCDWT^{-1}(s_0.L^1, s_2.L^2, s_4.L^1), \\
    s_1.L^1 = MCDWT^{-1}(s_0.L^1, s_1.L^2, s_2.L^1), \\
    s_3.L^1 = MCDWT^{-1}(s_2.L^1, s_3.L^2, s_4.L^1).
  $
  
* Scale 0:

  $
    s_0.L^0 = DWT^{-1}(s_0.L^1, s_0.H^1),\\
    s_4.L^0 = DWT^{-1}(s_4.L^1, s_4.H`1),\\
    s_2.L^0 = MCDWT^{-1}(s_0.L^0, s_2.L^1, s_4.L^0),\\
    s_1.L^0 = MCDWT^{-1}(s_0.L^0, s_1.L^1, s_2.L^0),\\
    s_3.L^0 = MCDWT^{-1}(s_2.L^0, s_3.L^2, s_4.L^0).
  $

  
Provided by the execution of iMCDWT.

1. Scale 2:

Provided by subbands `L` of the pyramids. We don't need to carry
out any computation.

2. Scale 1:

Rendered after running iMCDWT one iteration. For 3 pyramids
`A={A.L,A.H}`, `B={B.L,~B.H}` and `C={C.L,C.H}`
where the subband `L` is the scale 2, the scale 1 is
recostructed by (see Algorithm iMCDWT_step):

```python
  x = 2**2 = 4
  [A.L] = 2D_iDWT(A.L,0);                              > Interpolate low-freqs A.L of V[0]
  [A.H] = 2D_iDWT(0,A.H);                              > Interpolate high-freqs A.H of V[0]
  V[0] = [A.L] + [A.H];                                > Reconstruct V[0] at spatial level 1
  [B.L] = 2D_iDWT(V[1].L,0);                           > 
  [~B.H] = 2D_iDWT(0,V[1].H);
  [C.L] = 2D_iDWT(V[2].L,0);
  [C.H] = 2D_iDWT(0,V[2].H);
  V[2] = [C.L] + [C.H] 
  [B.L]->[A.L] = ME([B.L], [A.L])
  [B.L]->[C.L] = ME([B.L], [C.L])
  [B.H]_A = MC([A.H], [B.L]->[A.L])
  [B.H]_C = MC([C.H], [B.L]->[C.L])
  [B.H] = [~B.H] + int(round(([B.H]_A + [B.H]_C)/2.0))
  V[1] = [B.L] + [B.H]
  [A.L] = [C.L]
  [A.H] = [C.H]
```

3. Scale 0:

Repeat the previous computations.

4. Scale -1:

Repeat the previous computations, placing 0's in the H subbands.

Temporal scalability
--------------------

Provided by the pruned execution of iMCDWT. Depending on the index of
the image to render, a number of images, that ranges between :math:`1`
(the best case) and :math:`1+l` (the worst case), are decoded.

Quality scalability
-------------------

Provided by partially reconstructing a set of coefficients selected by
their contribution to the minimization of the distortion.

A sequence `S` of `N` (wavelet) pyramids, organized in `T` temporal subbands, where each temporal subband is a sequence of spatial pyramids. The number of input and output pyramids is the same.

For example, if `l=2` and `n=5`:

```
      Spatial
      scale 0 1 2       t = 1                               t = 3
            ^ ^ ^ +---+---+-------+                   +---+---+-------+                                ^
            | | | |   |   |       |                   |   |   |       |                                |
            | | v +---+---+       |                   +---+---+    O <---- T[3][y][x]                  |
            | |   |   |   |       |                   |   |   |       |                                |
            | v   +---+---+-------+                   +---+---+-------+ l = 0                          |
            |     |       |       |                   |       |       |                                |
            |     |       |       |                   |       |       |                                |
            |     |       |       |                   |       |       |                                |
            v     +-------+-------+       t = 2       +-------+-------+                                |
                      |       |     +---+---+-------+     |        |                                 ^ |
                      |       |     |   |   |       |     |        |                                 | |
                      |       +---->+---+---+       |<----+        |                                 | |
                      |             |   |   |       |              |                                 | |
                      |             +---+---+-------+ l = 1        |                                 | |
                      |             |       |       |              |                                 | |
                      |             |       |       |              |                                 | |
                      |             |       |       |              |                                 | |
      t = 0           |             +-------+-------+              |           t = 4                 | |
+---+---+-------+     |                 |       |                  |     +---+---+-------+         ^ | |
|   |   |       |     |                 |       |                  |     |   |   |       |         | | |
+---+---+       |<----+                 |       |                  +---->+---+---+       |         | | |
|   |   |       |                       |       |                        |   |   |       |         | | |
+---+---+-------+                       |       |                        +---+---+-------+  l = 2  | | |
|       |       |                       |       |                        |       |       |         | | |
|       |       |<----------------------+       +----------------------->|       |       |         | | |
|       |       |                                                        |       |       |         | | |
+-------+-------+                                                        +-------+-------+         v v v
      GOP 0                                       GOP 1                             Temporal scale 2 1 0
<---------------><----------------------------------------------------------------------->

(X --> Y) = X depends on Y (X has been encoded using Y)
```

## A implementation

### Forward MCDWT

```python
n = 5 # Number of frames of the video
l = 2 # Number of temporal scales to generate

x = 2 # A constant
for j in range(l):
    2D_DWT(V[0]) # 1-level 2D-DWT
    [A.L] = 2D_iDWT(V[0].L, 0)
    [A.H] = 2D_iDWT(0, V[0].H)
    i = 0 # Image index
    while i < (n//x):
        2D_DWT(V[x*i+x//2])
        [B.L] = 2D_iDWT(V[x*i+x//2].L, 0)
        [B.H] = 2D_iDWT(0, V[x*i+x//2].H)
        2D_DWT(V[x*i+x])
        [C.L] = 2D_iDWT(V[x*i+x].L, 0)
        [C.H] = 2D_iDWT(0, V[x*i+x].H)
        [B.L]->[A.L] = ME([B.L], [A.L])
        [B.L]->[C.L] = ME([B.L], [C.L])
        [B.H]_A = MC([A.H], [B.L]->[A.L])
        [B.H]_C = MC([C.H], [B.L]->[C.L])
        [~B.H] = [B.H] - int(round(([B.H]_A + [B.H]_C)/2.0))
        2D_DWT([~B.H])
        [~B.H].L = B.L
        [A.L] = [C.L]
        [A.H] = [C.H]
        i += 1
    x *= 2
```

Example (3 temporal scales (`l=2` iterations of the transform) and `n=5` images):
```
V[0] V[1] V[2] V[3] V[4]
 A    B    C              <- First call of MCDWT_step
           A    B    C    <- Second call of MCDWT_step
 A         B         C    <- Third call of MCDWT_step
---- -------------------
GOP0        GOP1
```

In [16]:
# %load ../src/color_dwt.py
import cv2
import numpy as np
import pywt
import math

def _2D_DWT(image):
    '''2D DWT of a color image.

    Arguments
    ---------

        image : [:,:,:].

            A color frame.

    Returns
    -------

        (L, H) where L = [:,:,:] and
        H = (LH, HL, HH), where LH, HL, HH = [:,:,:].

            A color pyramid.

    '''

    y = math.ceil(image.shape[0]/2)
    x = math.ceil(image.shape[1]/2)
    LL = np.ndarray((y, x, 3), np.float64)
    LH = np.ndarray((y, x, 3), np.float64)
    HL = np.ndarray((y, x, 3), np.float64)
    HH = np.ndarray((y, x, 3), np.float64)
    for c in range(3):
        (LL[:,:,c], (LH[:,:,c], HL[:,:,c], HH[:,:,c])) = pywt.dwt2(image[:,:,c], 'db5', mode='per')

    return (LL, (LH, HL, HH))

def _2D_iDWT(L, H):
    '''2D 1-iteration inverse DWT of a color pyramid.

    Arguments
    ---------

        L : [:,:,:].

            Low-frequency color subband.

        H : (LH, HL, HH), where LH ,HL, HH = [:,:,:].

            High-frequency color subbands.

    Returns
    -------

        [:,:,:].

            A color frame.

    '''

    LH = H[0]
    HL = H[1]
    HH = H[2]
    frame = np.ndarray((L.shape[0]*2, L.shape[1]*2, 3), np.float64)
    for c in range(3):
        frame[:,:,c] = pywt.idwt2((L[:,:,c], (LH[:,:,c], HL[:,:,c], HH[:,:,c])), 'db5', mode='per')
    return frame


In [27]:
# %load ../src/image_io.py
import cv2
import numpy as np

class InputFileException(Exception):
    pass

class ImageReader:
    '''Read 16-bit PNG images from disk.
    
    Images must be enumerated (image000.png, image001.png, ...).

    '''

    def __init__(self):
        pass

    def read(self, number, path='./'):
        '''Read an image from disk.

        Parameters
        ----------

            number : int.

                Index of the image in the sequence.

            path : str.

                Image path.

        Returns
        -------

            [:,:,:].

                A color image.

        '''

        file_name = '{}{:03d}.png'.format(path, number)
        image = cv2.imread(file_name, -1)
        if image is None:
            raise InputFileException('{} not found'.format(file_name))
        else:
            image -= 32768
            return image

class ImageWritter:
    '''Write 16-bit PNG images to disk.

    Images should be enumerated (image000.png, image001.png, ...).

    '''

    def __init__(self):
        pass

    def write(self, image, number=0, path='./'):
        '''Write an image to disk.

        Parameters
        ----------

            image : [:,:,:].

                The color image to write.

            number : int.

                Index of the image in the sequence.

            path : str.

                Path to the image.

        Returns
        -------

            None.

        '''

        file_name = '{}{:03d}.png'.format(path, number)

        image += 32768
        
        assert (np.amax(image) < 65536), '16 bit unsigned int range overflow'
        assert (np.amin(image) >= 0), '16 bit unsigned int range underflow'
        
        cv2.imwrite(file_name, np.rint(image).astype(np.uint16))


In [28]:
# %load ../src/motion_compensation.py
import numpy as np
import cv2

def motion_compensation(curr, next, base):
    flow = motion_estimation(curr, next)
    return estimate_frame(base, flow)

def motion_estimation(curr, next):
    curr_y, _, _ = cv2.split(curr)
    next_y, _, _ = cv2.split(next)

    return cv2.calcOpticalFlowFarneback(next_y, curr_y, 
            None, 0.5, 3, 15, 3, 5, 1.2, 0)

def estimate_frame(base, flow):
    height, width = flow.shape[:2]
    map_x = np.tile(np.arange(width), (height, 1))
    map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
    map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')

    return cv2.remap(base, map_xy, None, 
            interpolation=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_REPLICATE)


In [29]:
import sys
sys.path.append("../src")

In [30]:
# %load ../src/mcdwt_transform.py
import cv2
import numpy as np
import pywt
import math

import image_io
import pyramid_io
import motion_compensation

def forward(input = '../images/', output='/tmp/', n=5, l=2):
    '''A Motion Compensated Discrete Wavelet Transform.

    Compute the 1D-DWT along motion trajectories. The input video (as
    a sequence of images) must be stored in disk (<input> directory)
    and the output (as a sequence of DWT coefficients that are called
    pyramids) will be stored in disk (<output> directory). So, this
    MCDWT implementation does not transform the video on the fly.

    Arguments
    ---------

        input : str

            Path where the input images are. Example:
            "../input/image".

        output : str

            Path where the (transformed) pyramids will be. Example:
            "../output/pyramid".

         n : int

            Number of images to process.

         l : int

            Number of leves of the MCDWT (temporal scales). Controls
            the GOP size. Examples: `leves`=0 -> GOP_size = 1, `leves`=1 ->
            GOP_size = 2, `levels`=2 -> GOP_size = 4. etc.

    Returns
    -------

        None.

    '''
    
    #import ipdb; ipdb.set_trace()
    ir = image_io.ImageReader()
    iw = image_io.ImageWritter()
    pw = pyramid_io.PyramidWritter()
    x = 2
    for j in range(l): # Number of temporal scales
        A = ir.read(0, input)
        tmpA = _2D_DWT(A)
        L_y = tmpA[0].shape[0]
        L_x = tmpA[0].shape[1]
        pw.write(tmpA, 0, output)        
        zero_L = np.zeros(tmpA[0].shape, np.float64)
        zero_H = (zero_L, zero_L, zero_L)
        AL = _2D_iDWT(tmpA[0], zero_H)
        iw.write(AL, 1)
        AH = _2D_iDWT(zero_L, tmpA[1])
        iw.write(AH, 1)
        i = 0
        while i < (n//x):
            B = ir.read(x*i+x//2, input)
            tmpB = _2D_DWT(B)
            BL = _2D_iDWT(tmpB[0], zero_H)
            BH = _2D_iDWT(zero_L, tmpB[1])
            C = ir.read(x*i+x, input)
            tmpC = _2D_DWT(C)
            pw.write(tmpC, x*i+x, output)
            CL = _2D_iDWT(tmpC[0], zero_H)
            CH = _2D_iDWT(zero_L, tmpC[1])
            BHA = motion.motion_compensation(BL, AL, AH)
            BHC = motion.motion_compensation(BL, CL, CH)
            iw.write(BH, x*i+x//2, output+'predicted')
            prediction = (BHA + BHC) / 2
            iw.write(prediction+128, x*i+x//2, output+'prediction')
            rBH = BH - prediction
            iw.write(rBH, x*i+x//2, output+'residue')
            rBH = _2D_DWT(rBH)
            #import ipdb; ipdb.set_trace()
            pw.write(rBH, x*i+x//2 + 1000)
            rBH[0][0:L_y,0:L_x,:] = tmpB[0]
            pw.write(rBH, x*i+x//2, output)
            AL = CL
            AH = CH
            i += 1
            print('i =', i)
        x *= 2

def backward(input = '/tmp/', output='/tmp/', n=5, l=2):
    '''A (Inverse) Motion Compensated Discrete Wavelet Transform.

    iMCDWT is the inverse transform of MCDWT. Inputs a sequence of
    pyramids and outputs a sequence of images.

    Arguments
    ---------

        input : str

            Path where the input pyramids are. Example:
            "../input/image".

        output : str

            Path where the (inversely transformed) images will
            be. Example: "../output/pyramid".

         n : int

            Number of pyramids to process.

         l : int

            Number of leves of the MCDWT (temporal scales). Controls
            the GOP size. Examples: `l`=0 -> GOP_size = 1, `l`=1 ->
            GOP_size = 2, `l`=2 -> GOP_size = 4. etc.

    Returns
    -------

        None.

    '''
    
    #import ipdb; ipdb.set_trace()
    ir = image_io.ImageReader()
    iw = image_io.ImageWritter()
    pr = pyramid_io.PyramidReader()
    x = 2**l
    for j in range(l): # Number of temporal scales
        #import ipdb; ipdb.set_trace()
        A = pr.read(0, input)
        zero_L = np.zeros(A[0].shape, np.float64)
        zero_H = (zero_L, zero_L, zero_L)
        AL = _2D_iDWT(A[0], zero_H)
        AH = _2D_iDWT(zero_L, A[1])
        A = AL + AH
        iw.write(A, 0)
        i = 0
        while i < (n//x):
            B = pr.read(x*i+x//2, input)
            BL = _2D_iDWT(B[0], zero_H)
            rBH = _2D_iDWT(zero_L, B[1])
            C = pr.read(x*i+x, input)
            CL = _2D_iDWT(C[0], zero_H)
            CH = _2D_iDWT(zero_L, C[1])
            C = CL + CH
            iw.write(C, x*i+x, output)
            BHA = motion.motion_compensation(BL, AL, AH)
            BHC = motion.motion_compensation(BL, CL, CH)
            BH = rBH + (BHA + BHC) / 2
            B = BL + BH
            iw.write(B, x*i+x//2, output)
            AL = CL
            AH = CH
            i += 1
            print('i =', i)
        x //=2


In [31]:
forward()

NameError: name 'np' is not defined

### iMCDWT code

```python
n = 5 # Number of images
l = 2 # Number of temporal scales

x = 2**l
for j in range(l):
    [A.L] = 2D_iDWT(V[0].L, 0)
    [A.H] = 2D_iDWT(0, V[0].H)
    V[0] = [A.L] + [A.H]
    i = 0 # Image index
    while i < (n//x):
        [B.L] = 2D_iDWT(V[x*i+x//2].L, 0)
        [~B.H] = 2D_iDWT(0, V[x*i+x//2].H)
        [C.L] = 2D_iDWT(V[x*i+x].L, 0)
        [C.H] = 2D_iDWT(0, V[x*i+x].H)
        V[x*i+x] = [C.L] + [C.H]
        [B.L]->[A.L] = ME([B.L], [A.L])
        [B.L]->[C.L] = ME([B.L], [C.L])
        [B.H]_A = MC([A.H], [B.L]->[A.L])
        [B.H]_C = MC([C.H], [B.L]->[C.L])
        [B.H] = [~B.H] + int(round(([B.H]_A + [B.H]_C)/2.0))
        V[x*i+x//2] = [B.L] + [B.H]
        [A.L] = [C.L]
        [A.H] = [C.H]
        i += 1
    x //= 2
```

Note: $x.L^n = \{x.L^{n+1}, x.H^{n+1}\}$